In [ ]:
import numpy as np
# import torch 
import terratorch

In [16]:
help(terratorch.models)

Help on package terratorch.models in terratorch:

NAME
    terratorch.models - # Copyright contributors to the Terratorch project

PACKAGE CONTENTS
    backbones (package)
    clay_model_factory
    decoders (package)
    generic_unet_model_factory
    heads (package)
    model
    pixel_wise_model
    prithvi_model_factory
    satmae_model_factory
    scalar_output_model
    scalemae_model_factory
    smp_model_factory
    timm_model_factory

CLASSES
    terratorch.models.model.ModelFactory(abc.ABC)
        terratorch.models.generic_unet_model_factory.GenericUnetModelFactory
        terratorch.models.prithvi_model_factory.PrithviModelFactory
        terratorch.models.satmae_model_factory.SatMAEModelFactory
        terratorch.models.scalemae_model_factory.ScaleMAEModelFactory
        terratorch.models.smp_model_factory.SMPModelFactory
        terratorch.models.timm_model_factory.TimmModelFactory
    
    class GenericUnetModelFactory(terratorch.models.model.ModelFactory)
     |  Method